In [ ]:
!pip install psycopg2-binary
!pip install -qU llama-index==0.9.29


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import psycopg2
import pandas as pd
from tqdm import tqdm
import json
import os
import chardet

# Accessing the Database

In [ ]:


# ngrok-provided settings
host = ""       # ngrok host from the forwarding URL
port = ""                # ngrok port from the forwarding URL

# PostgreSQL credentials
user = ""
password = ""
database = ""


# Connect to PostgreSQL
try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=database
    )
    print("PSQL Connection successful!")
except Exception as e:
    print(f"Error connecting to database: {e}")



# Set display options
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Adjust width to avoid line wrapping
pd.set_option('display.max_colwidth', None)  # Display full content in each cell


# Execute Query
def exec_query(query):
    try:
        # Establish the connection
        conn = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=database
        )
        cursor = conn.cursor()

        # Execute the query
        cursor.execute(query)

        # Fetch column names from the cursor
        colnames = [desc[0] for desc in cursor.description]

        # Fetch all rows of the query
        results = cursor.fetchall()

        # Create a DataFrame with column names
        df = pd.DataFrame(results, columns=colnames)

        # Close the cursor and connection
        cursor.close()
        conn.close()

        # Return the DataFrame
        return df

    except Exception as e:
        print(f"Error executing query: {e}")
        # Roll back the transaction in case of error
        conn.rollback()
        cursor.close()
        conn.close()
        return None

# Create Views from the complete data

In [ ]:
list_tables_query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
"""

list_views_query = """
SELECT table_name
FROM information_schema.views
WHERE table_schema = 'public';
"""

exec_query(list_views_query)


# 1. Patient Demographics and Admissions
query1 = """
CREATE OR REPLACE VIEW admissions_view AS
SELECT DISTINCT
    p.subject_id,
    p.gender,
    p.anchor_age,
    p.anchor_year_group,
    p.dod AS date_of_death,
    a.hadm_id,
    a.admission_type,
    a.race,
    a.hospital_expire_flag
FROM
    patients AS p
INNER JOIN
    admissions AS a ON p.subject_id = a.subject_id;
"""


# 2. Patient Diagnoses
query2 = """
CREATE OR REPLACE VIEW diagnoses_view AS
SELECT DISTINCT
    d.subject_id,
    d.hadm_id,
    d.icd_code,
    dicd.long_title AS diagnosis_description
FROM
    diagnoses_icd AS d
INNER JOIN
    d_icd_diagnoses AS dicd
    ON d.icd_code = dicd.icd_code AND d.icd_version = dicd.icd_version
"""


# 3. Patient Procedures
query3 = """
CREATE OR REPLACE VIEW procedures_view AS
SELECT DISTINCT
    pr.subject_id,
    pr.hadm_id,
    pr.icd_code,
    dicp.long_title AS procedure_description
FROM
    procedures_icd AS pr
INNER JOIN
    d_icd_procedures AS dicp
    ON pr.icd_code = dicp.icd_code AND pr.icd_version = dicp.icd_version
"""


# 4. Medication Orders
query4 = """
CREATE OR REPLACE VIEW medications_view AS
SELECT DISTINCT
    pr.subject_id,
    pr.hadm_id,
    pr.drug,
    pr.dose_val_rx AS dose_value,
    pr.dose_unit_rx AS dose_unit,
    pr.route
FROM
    prescriptions AS pr
WHERE
    pr.subject_id IS NOT NULL
"""


# 5. Laboratory Results
query5 = """
CREATE OR REPLACE VIEW lab_results_view AS
SELECT DISTINCT
    le.subject_id,
    le.hadm_id,
    dl.label AS test_name,
    le.valuenum AS test_value,
    le.valueuom AS test_unit,
    le.flag AS abnormal_flag
FROM
    labevents AS le
INNER JOIN
    d_labitems AS dl ON le.itemid = dl.itemid
WHERE
    le.valuenum IS NOT NULL
"""



# 6. Microbiology Results
query7 = """
CREATE OR REPLACE VIEW microbiology_results_view AS
SELECT DISTINCT
    me.subject_id,
    me.hadm_id,
    me.spec_type_desc AS specimen_type,
    me.test_name,
    me.org_name AS organism_name,
    me.ab_name AS antibiotic_name,
    me.interpretation,
    me.comments
FROM
    microbiologyevents AS me
WHERE
    me.subject_id IS NOT NULL
    AND me.hadm_id IS NOT NULL
"""



# List of view names
view_names = [
    'medications_view',
    'lab_results_view',
    'admissions_view',
    'diagnoses_view',
    'procedures_view',
    'microbiology_results_view'
]
# Loop through each view and display 5 records
for view in view_names:
    query = f"SELECT * FROM {view} LIMIT 5;"
    df = exec_query(query)
    print(f" {view}:")
    display(df)
    print("\n")




# Structured Data -> Semi-Structured JSON

In [ ]:

# ngrok-provided settings
host = ""       # ngrok host from the forwarding URL
port = ""                # ngrok port from the forwarding URL

# PostgreSQL credentials
user = ""
password = ""
database = ""


# Establish the connection
conn = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=database
    )

# Initialize the cursor
cursor = conn.cursor()

# Define queries for each view
queries = {
    "admissions_view": "SELECT * FROM admissions_view;",
    "diagnoses_view": "SELECT * FROM diagnoses_view;",
    "procedures_view": "SELECT * FROM procedures_view;",
    "medications_view": "SELECT * FROM medications_view;",
    "lab_results_view": "SELECT * FROM lab_results_view;",
    "microbiology_results_view": "SELECT * FROM microbiology_results_view;"
}

# Initialize a dictionary to hold unstructured data by subject_id
data_by_subject_id = {}

# Process each view and aggregate data by subject_id and hadm_id
for view_name, query in queries.items():
    cursor.execute(query)
    columns = [desc[0] for desc in cursor.description]  # Column names
    rows = cursor.fetchall()

    for row in rows:
        record = dict(zip(columns, row))  # Convert row to dictionary
        subject_id = record["subject_id"]
        hadm_id = record["hadm_id"]

        if subject_id not in data_by_subject_id:
            data_by_subject_id[subject_id] = {
                "sr_id": None,  # Will be assigned later
                "hadm_ids": {}
            }

        # Initialize structure for hadm_id within the subject
        if hadm_id not in data_by_subject_id[subject_id]["hadm_ids"]:
            data_by_subject_id[subject_id]["hadm_ids"][hadm_id] = {
                "admissions": {},
                "medications": [],
                "lab_results": [],
                "diagnoses": [],
                "procedures": [],
                "microbiology_results": []
            }

        # Organize records based on view into appropriate fields
        if view_name == "admissions_view":
            data_by_subject_id[subject_id]["hadm_ids"][hadm_id]["admissions"].update({
                "gender": record["gender"],
                "anchor_age": record["anchor_age"],
                "anchor_year_group": record["anchor_year_group"],
                "date_of_death": record["date_of_death"],
                "admission_type": record["admission_type"],
                "race": record["race"],
                "hospital_expire_flag": record["hospital_expire_flag"]
            })
        elif view_name == "diagnoses_view":
            data_by_subject_id[subject_id]["hadm_ids"][hadm_id]["diagnoses"].append({
                "icd_code": record["icd_code"],
                "description": record["diagnosis_description"]
            })
        elif view_name == "procedures_view":
            data_by_subject_id[subject_id]["hadm_ids"][hadm_id]["procedures"].append({
                "icd_code": record["icd_code"],
                "description": record["procedure_description"]
            })
        elif view_name == "medications_view":
            data_by_subject_id[subject_id]["hadm_ids"][hadm_id]["medications"].append({
                "drug": record["drug"],
                "dose": record["dose_value"],
                "dose_unit": record["dose_unit"],
                "route": record["route"]
            })
        elif view_name == "lab_results_view":
            data_by_subject_id[subject_id]["hadm_ids"][hadm_id]["lab_results"].append({
                "test_name": record["test_name"],
                "test_value": record["test_value"],
                "test_unit": record["test_unit"],
                "abnormal_flag": record["abnormal_flag"]
            })
        elif view_name == "microbiology_results_view":
            data_by_subject_id[subject_id]["hadm_ids"][hadm_id]["microbiology_results"].append({
                "specimen_type": record["specimen_type"],
                "test_name": record["test_name"],
                "organism_name": record.get("organism_name"),
                "antibiotic_name": record.get("antibiotic_name"),
                "interpretation": record.get("interpretation"),
                "comments": record["comments"]
            })

cursor.close()
conn.close()

# Initialize the sr_id counter
sr_id_counter = 1001

# Prepare the final unstructured data
semistructured_data = []

# Iterate over each subject_id to convert data into the desired JSON format
for subject_id, subject_data in tqdm(data_by_subject_id.items(), desc="Processing subjects"):
    # Assign a unique sr_id to each subject
    subject_data["sr_id"] = sr_id_counter
    sr_id_counter += 1  # Increment the unique id for each entry

    # Append the structured data to the final unstructured data list
    semistructured_data.append({
        "sr_id": subject_data["sr_id"],
        "subject_id": subject_id,
        "hadm_ids": subject_data["hadm_ids"]
    })


# Save to JSON file
output_file = "/content/drive/MyDrive/Research/PSQL-MIMIC-IV/MIMIC-IV-Semi-Structured-Data-Hierarchical.json"
with open(output_file, 'w') as f:
    json.dump(semistructured_data, f, indent=2)

# Reducing the JSON

In [ ]:
# REDUCING JSON

# Mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive')

# Path to the input JSON file
input_file = "/content/drive/MyDrive/Research/PSQL-MIMIC-IV/MIMIC-IV-Semi-Structured-Data-Hierarchical.json"

# Path to the output JSON file
output_file = "/content/drive/MyDrive/Research/PSQL-MIMIC-IV/MIMIC-IV-Semi-Structured-Data-Hierarchical-Reduced.json"

# Read the JSON file
with open(input_file, 'r') as f:
    data = json.load(f)

# Iterate over the data and modify as per the requirements
for record in data:
    # For each record, get hadm_ids and limit to 5 hadm_ids
    hadm_ids = record.get('hadm_ids', {})
    # Convert hadm_ids to a list of tuples (hadm_id, hadm_data)
    hadm_items = list(hadm_ids.items())
    # Limit to first 5 hadm_ids
    limited_hadm_items = hadm_items[:5]
    # Create a new hadm_ids dict with limited hadm_ids
    new_hadm_ids = {}
    for hadm_id, hadm_data in limited_hadm_items:
        # Truncate medications to first 5 records
        if 'medications' in hadm_data:
            hadm_data['medications'] = hadm_data['medications'][:5]
        # Truncate lab_results to first 5 records
        if 'lab_results' in hadm_data:
            hadm_data['lab_results'] = hadm_data['lab_results'][:5]
        # Add the modified hadm_data back to new_hadm_ids
        new_hadm_ids[hadm_id] = hadm_data
    # Update the record's hadm_ids with the new limited hadm_ids
    record['hadm_ids'] = new_hadm_ids

# Save the modified data back to JSON
with open(output_file, 'w') as f:
    json.dump(data, f, indent=2)

print(f"Reduced JSON saved to {output_file}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reduced JSON saved to /content/drive/MyDrive/Research/PSQL-MIMIC-IV/MIMIC-IV-Semi-Structured-Data-Hierarchical-Reduced.json


# JSON -> Text for Indexing

In [ ]:


# Load your hierarchical JSON data
input_file = "/content/drive/MyDrive/Research/PSQL-MIMIC-IV/MIMIC-IV-Semi-Structured-Data-Hierarchical-Reduced.json"
with open(input_file, 'r') as f:
    unstructured_data = json.load(f)

# Function to convert each patient's data to a structured text format
def convert_to_text(data):
    text_data = []
    for entry in tqdm(data, desc="Processing entries"):
        subject_text = f"Subject ID: {entry['subject_id']} (SR ID: {entry['sr_id']})\n"

        for hadm_id, hadm_data in entry["hadm_ids"].items():
            hadm_text = f"\n---\n\nAdmission ID: {hadm_id}\n"

            # Admission details
            if hadm_data["admissions"]:
                admission = hadm_data["admissions"]
                admission_text = (
                    f"Admission Details:\n"
                    f"  1. Gender: {admission['gender']}\n"
                    f"  2. Age: {admission['anchor_age']}\n"
                    f"  3. Admission Year Group: {admission['anchor_year_group']}\n"
                    f"  4. Admission Type: {admission['admission_type']}\n"
                    f"  5. Race: {admission['race']}\n"
                    f"  6. Expired in Hospital: {'Yes' if admission['hospital_expire_flag'] == '1' else 'No'}\n"
                    f"  7. Date of Death: {admission['date_of_death'] if admission['date_of_death'] else 'Not Applicable'}\n"
                )
                hadm_text += admission_text

            # Medications
            medications = hadm_data["medications"]
            if medications:
                medication_text = "Medications:\n"
                for i, med in enumerate(medications, start=1):
                    medication_text += (
                        f"  {i}. {med['drug']}\n"
                        f"     - Dose: {med['dose']} {med['dose_unit']}\n"
                        f"     - Route: {med['route']}\n"
                    )
                hadm_text += medication_text

            # Lab Results
            lab_results = hadm_data["lab_results"]
            if lab_results:
                lab_text = "Lab Results:\n"
                for i, lab in enumerate(lab_results, start=1):
                    abnormality = " (Abnormal)" if lab["abnormal_flag"] == "abnormal" else ""
                    lab_text += (
                        f"  {i}. {lab['test_name']}\n"
                        f"     - Value: {lab['test_value']} {lab['test_unit']}{abnormality}\n"
                    )
                hadm_text += lab_text

            # Diagnoses
            diagnoses = hadm_data["diagnoses"]
            if diagnoses:
                diagnosis_text = "Diagnoses:\n"
                for i, diag in enumerate(diagnoses, start=1):
                    diagnosis_text += (
                        f"  {i}. ICD Code {diag['icd_code']}\n"
                        f"     - Description: {diag['description']}\n"
                    )
                hadm_text += diagnosis_text

            # Procedures
            procedures = hadm_data["procedures"]
            if procedures:
                procedure_text = "Procedures:\n"
                for i, proc in enumerate(procedures, start=1):
                    procedure_text += (
                        f"  {i}. ICD Code {proc['icd_code']}\n"
                        f"     - Description: {proc['description']}\n"
                    )
                hadm_text += procedure_text

            # Microbiology Results
            micro_results = hadm_data["microbiology_results"]
            if micro_results:
                micro_text = "Microbiology Results:\n"
                for i, micro in enumerate(micro_results, start=1):
                    comments = f" (Comments: {micro['comments']})" if micro["comments"] else ""
                    micro_text += (
                        f"  {i}. Specimen: {micro['specimen_type']}\n"
                        f"     - Test: {micro['test_name']}\n"
                        f"     - Interpretation: {micro['interpretation'] if micro['interpretation'] else 'Not Provided'}\n"
                        f"     {comments}\n"
                    )
                hadm_text += micro_text

            subject_text += hadm_text

        # Append the final structured text for this subject
        text_data.append(subject_text)
    return text_data

# Convert the unstructured data to structured text format
structured_text_data = convert_to_text(unstructured_data)

# Save to a text file for indexing with LlamaIndex
output_file = "/content/drive/MyDrive/Research/PSQL-MIMIC-IV/MIMIC-IV-Structured-Text-Data-Reduced.txt"
with open(output_file, 'w') as f:
    for text_entry in structured_text_data:
        f.write(text_entry + "\n\n")  # Add separation between entries


Processing entries: 100%|██████████| 100/100 [00:00<00:00, 7568.21it/s]


# Creating Indexes with VectorStoreIndex

In [ ]:


with open('/content/drive/MyDrive/Research/PSQL-MIMIC-IV/MIMIC-IV-Structured-Text-Data-Reduced.txt', 'rb') as file:
    raw_data = file.read()

result = chardet.detect(raw_data)
file_encoding = result['encoding']
print(file_encoding)

# Use the detected encoding
with open('/content/drive/MyDrive/Research/PSQL-MIMIC-IV/MIMIC-IV-Structured-Text-Data-Reduced.txt', 'r', encoding=file_encoding) as file:
    big_text = file.read()

import os
from llama_index import VectorStoreIndex, Document

# Set OpenAI API Key
os.environ['OPENAI_API_KEY'] = ''

# Load and split your encyclopedia text
sections = big_text.split('\n\n')  # Split by paragraphs or sections

# Filter out empty sections and create Document objects
documents = [Document(text=section) for section in sections if section.strip()]

# Check if documents list is empty after filtering
if not documents:
    raise ValueError("No content to index. Please ensure 'big_text' has content.")



ISO-8859-1


### Storing Indexes

In [ ]:
# Ensure the directory exists

persist_dir = "/content/drive/MyDrive/Research/PSQL-MIMIC-IV/VectorStore_Indexes"
os.makedirs(persist_dir, exist_ok=True)

In [ ]:
# Build your index with the default storage context
from llama_index import VectorStoreIndex

# Assume 'documents' is your list of Document objects
index = VectorStoreIndex.from_documents(documents)

# Save the index to the specified directory
index.storage_context.persist(persist_dir=persist_dir)

print(f"Index successfully saved to {persist_dir}")

Index successfully saved to /content/drive/MyDrive/Research/PSQL-MIMIC-IV/VectorStore_Indexes


### Load Indexes to use them

In [ ]:
# LOAD INDEXES

# Import necessary libraries
from llama_index import StorageContext, load_index_from_storage


# Set OpenAI API Key
os.environ['OPENAI_API_KEY'] = ''

# Specify the storage directory
persist_dir = "/content/drive/MyDrive/Research/PSQL-MIMIC-IV/VectorStore_Indexes"

# Initialize the storage context with the persist_dir
storage_context = StorageContext.from_defaults(persist_dir=persist_dir)

# Load the index from the storage context
index = load_index_from_storage(storage_context)

print("Index successfully loaded from storage.")

Index successfully loaded from storage.


# Sample Queries

In [ ]:
# Create a query engine from the index
query_engine = index.as_query_engine()

# Perform a query
response = query_engine.query("Which patients received an intravenous dose of Potassium Chloride, and what were their corresponding diagnoses?")

# Print the response
print(response)

Patient with Admission ID 21133938 received an intravenous dose of Potassium Chloride. The corresponding diagnoses for this patient were Hyperkalemia and Acute and subacute hepatic failure without coma.


In [ ]:
# Perform a query
response = query_engine.query("Which patients received a transfusion of 0.9% Sodium Chloride, and what were their vital lab test results post-administration?")

# Print the response
print(response)

Patient with Admission ID 27703517 received a transfusion of 0.9% Sodium Chloride. The vital lab test results post-administration were as follows:
- Eosinophils: 2.7%
- MCHC: 33.3%
- MCV: 83 fL
- Platelet Count: 335 K/uL
- Anion Gap: 16 mEq/L


In [ ]:
# Perform a query
response = query_engine.query("Identify patients with a diagnosis of portal hypertension and the procedures they underwent.")

# Print the response
print(response)

Patient with Admission ID 21636229 has a diagnosis of portal hypertension. The procedure they underwent was Drainage of Peritoneal Cavity, Percutaneous Approach, Diagnostic.

Patient with Admission ID 23514107 also has a diagnosis of portal hypertension. No specific procedure related to portal hypertension was mentioned in the provided context information.


In [ ]:
# Perform a query
response = query_engine.query("how to treat stage 3 pancreatic cancer")

# Print the response
print(response)

Surgery, chemotherapy, and radiation therapy are common treatments for stage 3 pancreatic cancer. In some cases, a combination of these treatments may be recommended to help manage the disease. It is important for patients to consult with their healthcare team to determine the most appropriate treatment plan based on their individual circumstances.
